Para importar as bibliotecas

In [4]:
import pandas as pd
import numpy as np
import qrcode
import os

Para gerar as pastas e os páginas HTML

In [5]:

base_html = pd.read_excel('finalizado.xlsx',
                              usecols=['Nome','Turma', 'Sala', 'Professor', 'Inscrição'])

# for i in range(len(base_html)):
for i in range(len(base_html)):
     
    site = f"""
    <!DOCTYPE html>
    <html lang="pt-br">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Germinare TECH</title>
        <style>
            @import url('https://fonts.googleapis.com/css2?family=Inter:wght@100..900&display=swap');

            * {{
                margin: 0;
                padding: 0;
                box-sizing: border-box;
                font-family: 'Inter', sans-serif;
                font-size: 25px;
            }}

            body {{
                background-color: #fdfdfd;
                color: #333;
                height: 100vh;
                padding: 1rem;
            }}

            h1 {{
                font-weight: 600;
                background-color: #9ed1e7;
                border-radius: .9rem;
                padding-block: .5rem;
                padding-inline: 1rem;
                display: inline-block;
            }}

            p {{
                margin-block: 1rem;
                font-size: 1rem;
            }}

            button {{
                outline: none;
                border: none;
                background-color: #9ed1e7;
                border-radius: .9rem;
                padding-block: .5rem;
                padding-inline: 1rem;
                font-weight: 450;
                cursor: pointer;
                transition: background-color .3s;
            }}

            button:hover {{
                background-color: #87b4c7;
            }}

            @media only screen and (max-width: 450px) {{
                h1 {{
                    font-size: 1rem;
                }}

                p {{
                    font-size: .8rem;
                }}
                button {{
                    font-size: .8rem;
                }}
            }}

            @media only screen and (max-width: 200px) {{
                h1 {{
                    font-size: .7rem;
                }}
                button {{
                    font-size: .6rem;
                }}
            }}
        
        </style>
    </head>
        <body>
        <h1>Informações do Candidato</h1>
            <p>
                Nome: <strong>{base_html["Nome"][i]}</strong>
            </p>
            <p>
                Turma: <strong>{base_html["Turma"][i]}</strong>
            </p>
            <p>
                Sala: <strong>{base_html["Sala"][i]}</strong>
            </p>
            <p>
                Professor: <strong>{base_html["Professor"][i]}</strong> 
            </p>
            
        </body>
        </html>
        """
    # print(info)


    folder = os.path.join("paginas_html/", base_html["Inscrição"][i])
    os.mkdir(folder)
    arquivo = open(f'{folder}/{base_html["Inscrição"][i]}.html', "a")
    #arquivo = open(f'teste.html', "a")

    # print(site)
    arquivo.write(site)
    arquivo.close()

Para gerar os QR_Codes

In [6]:
df_qr = pd.read_excel('finalizado.xlsx',
                              usecols=['Nome','Turma', 'Sala', 'Professor', 'Inscrição'])

# for i in range(len(df_qr)):
for i in range(len(df_qr)):
    nome_insc = ""
    link = f'https://gedaijef.github.io/qrcode/paginas_html/{df_qr["Inscrição"][i].replace(" ", "_")}/{df_qr["Inscrição"][i].replace(" ", "_")}.html'
    qrimg = qrcode.make(link)
    qrimg.save(f'qrCodes/{df_qr["Inscrição"][i].replace(" ", "_")}.png')